### The following code cell imports the necessary packages, creates the file connection URLs and connects to a SQLite database file (Creates one if it does not exist).

In [27]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
import numpy as np
import random
import sqlite3
from bokeh.io import curdoc
from bokeh.plotting import figure, output_file, show
mega_draw_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/megamillions.csv")
mega_weather_file_url = ("https://raw.githubusercontent.com/Mikdown/Lottery-Picker/master/assets/mega_weather.csv")

conn = sqlite3.connect('assets/lottery_data.db')
cur = conn.cursor()

### The following three code cells meet the first and second feature requirements:
1. Feature 1 - Read two data files (CSV).
2. Feature 2 - Clean your data.
    - Two CSV files are read in from Github and cleaned(transformed) with Pandas. 
3. Feature 1 - Set up a local database and read data in with SQLite.
4. Feature 2 - Perform a SQL join.
    - After transformation the files are read into a local SQLite3 DB to be used for futher analysis.
    - Data is read in from the database, filtered, grouped and a SQL join is performed on 2 tables.

In [3]:
#Create the variables for file reads.
mega_draw_file = (mega_draw_file_url)
mega_weather_file = (mega_weather_file_url)

#Read in data from two CSV files.
mega_draw_df = pd.read_csv(mega_draw_file, engine='python', parse_dates= {"draw_date" : ["year","month","day"]})
mega_weather_df = pd.read_csv(mega_weather_file, engine='python')

#Split the "DATE_TIME" column into 2 seperate columns "Date" and "Time".
mega_weather_df['Date'] = pd.to_datetime(mega_weather_df['DATE_TIME']).dt.strftime('%Y%m%d')
mega_weather_df['Time'] = pd.to_datetime(mega_weather_df['DATE_TIME']).dt.strftime('%H%M')

#Set the dtype to "datetime" for the two date coulmns.
mega_weather_df['Date'] = pd.to_datetime(mega_weather_df['Date'])
mega_weather_df['DATE_TIME'] = pd.to_datetime(mega_weather_df['DATE_TIME'])

#Clean/Transform - 1. Rename column, 
                  #2. Format date string, 
                  #3. Set dtype to "datetime", 
                  #4. Create 4 new columns from the "Date" column. 
                  #5. Set dtype to "int" for the number columns.
mega_draw_df.rename({'draw_date': 'Date'}, axis=1, inplace=True)
mega_draw_df['Date'] = pd.to_datetime(mega_draw_df['Date']).dt.strftime('%Y%m%d')
mega_draw_df['Date'] = pd.to_datetime(mega_draw_df['Date'])
mega_draw_df['month'] = pd.DatetimeIndex(mega_draw_df['Date']).month_name()
mega_draw_df['day_name'] = pd.DatetimeIndex(mega_draw_df['Date']).day_name()
mega_draw_df['day_num'] = pd.DatetimeIndex(mega_draw_df['Date']).dayofweek
mega_draw_df['day_date'] = pd.DatetimeIndex(mega_draw_df['Date']).day
mega_draw_df['draw_time'] = '2300'
mega_draw_df[['num_1', 'num_2', 'num_3', 'num_4', 'num_5', 'mb']] = mega_draw_df[['num_1', 'num_2', 'num_3', 'num_4', 'num_5', 'mb']].astype(int)

#Create two new tables in the SQLite DB file.
mega_draw_df.to_sql('mm_draw', conn, if_exists='replace', index = False)
mega_weather_df.to_sql('mm_weather', conn, if_exists='replace', index = False)

#Print the outputs of the dataframes to confirm changes have been applied.
print(mega_draw_df)
print(mega_weather_df)

           Date           game  num_1  num_2  num_3  num_4  num_5  mb     month day_name  day_num  day_date draw_time
0    2003-12-05  Mega Millions     12     44     15     18      1  42  December   Friday        4         5      2300
1    2003-12-09  Mega Millions     14     15     48      4     24  41  December  Tuesday        1         9      2300
2    2003-12-12  Mega Millions     16     32     46      9     45  26  December   Friday        4        12      2300
3    2003-12-16  Mega Millions     47     16     31     24     46  47  December  Tuesday        1        16      2300
4    2003-12-19  Mega Millions      5     10     39     17     35  38  December   Friday        4        19      2300
...         ...            ...    ...    ...    ...    ...    ...  ..       ...      ...      ...       ...       ...
2006 2023-02-24  Mega Millions     22     49      2     65     67   7  February   Friday        4        24      2300
2007 2023-02-28  Mega Millions     59     52     40     

In [4]:
#Read in and perform a SQL FILTER query on the weather table in the DB. 
    #The weather table contains hourly readings for a 24 hour period and I only
    #want the data that is within 1 hour +- the "draw_time".
mm_sql_filter = '''SELECT * FROM mm_weather WHERE (Time > 2200 AND Time < 2359)'''

#Create a dataframe with the filtered query data.
mm_weather_filtered_df = pd.read_sql(mm_sql_filter, conn)

#Clean/Transform 1. Set dtype to "datetime" for the date columns.
                #2. Fill empty cells with 0s.
mm_weather_filtered_df['DATE_TIME'] = pd.to_datetime(mm_weather_filtered_df['DATE_TIME'])
mm_weather_filtered_df['Date'] = pd.to_datetime(mm_weather_filtered_df['Date'])
mm_weather_filtered_df = mm_weather_filtered_df.fillna(0)

#Create a new SQLite DB table with the filtered dataframe.
mm_weather_filtered_df.to_sql('mm_weather_filtered', conn, if_exists='replace', index = False)

#Read in and perform a SQLite GROUP BY query in order to reduce the number of filtered weather
    #reports to 1 line.
mm_sql_grooper = '''SELECT * FROM mm_weather_filtered GROUP BY Date'''

#Create a dataframe with the grouped query data.
mm_weather_grouped_df = pd.read_sql(mm_sql_grooper, conn)

#Create a new SQLite DB table with the grouped dataframe.
mm_weather_grouped_df.to_sql('mm_weather_grouped', conn, if_exists='replace', index = False)

#Perform a SQL JOIN on the mm_draw table and the mm_weather_grouped table to combine
#all of the data associated with a single matching date in each table.
mm_sql_join = '''SELECT * FROM mm_draw JOIN mm_weather_grouped USING (Date)'''

#Create a dataframe with the join data.
mm_join_df = pd.read_sql(mm_sql_join, conn)

#Create a new SQLite DB table with the join dataframe.
mm_join_df.to_sql('mm_join', conn, if_exists='replace', index = False)

#Print the outputs of the dataframes to confirm changes have been applied.
print(mm_weather_filtered_df)
print(mm_weather_grouped_df)
print(mm_join_df)

               DATE_TIME DPTemp  DBTemp  Precip  RHumid       Date  Time
0    2013-01-01 22:53:00     50      51    0.06      96 2013-01-01  2253
1    2013-01-01 23:53:00     50      51    0.07      96 2013-01-01  2353
2    2013-01-02 22:53:00     35      40    0.00      83 2013-01-02  2253
3    2013-01-02 23:53:00     36      39    0.00      89 2013-01-02  2353
4    2013-01-03 22:53:00     27      36    0.00      70 2013-01-03  2253
...                  ...    ...     ...     ...     ...        ...   ...
8981 2022-12-14 23:53:00     46      48    0.16      93 2022-12-14  2353
8982 2022-12-15 22:53:00     32      44    0.00      63 2022-12-15  2253
8983 2022-12-15 23:53:00     32      44    0.00      63 2022-12-15  2353
8984 2022-12-16 22:53:00     28      43    0.00      56 2022-12-16  2253
8985 2022-12-16 23:53:00     27      44    0.00      51 2022-12-16  2353

[8986 rows x 7 columns]
                DATE_TIME DPTemp  DBTemp  Precip  RHumid                 Date  Time
0     2013-01-0

In [13]:
#Count the number of times each value appears in each column.
mm_counts_1 = mega_draw_df['num_1'].value_counts()
mm_counts_2 = mega_draw_df['num_2'].value_counts()
mm_counts_3 = mega_draw_df['num_3'].value_counts()
mm_counts_4 = mega_draw_df['num_4'].value_counts()
mm_counts_5 = mega_draw_df['num_5'].value_counts()
mm_counts_mb = mega_draw_df['mb'].value_counts()

#Create a dataframe with columns containing the counts of each value.
mm_counts_df = pd.concat([mm_counts_1, mm_counts_2, mm_counts_3, mm_counts_4, mm_counts_5, mm_counts_mb], axis=1)

#Fill empty cells with 0s and set dtype "int".
mm_counts_df['mb'] = mm_counts_df['mb'].fillna(0).astype(int)

#Create a list with all of the columns from the mm_counts_df dataframe.
col_list= list(mm_counts_df)

#Remove the "mb" column from the list as it is already totaled and should not be included.
col_list.remove('mb')

#Create a new column with the sum of the totals of each value in each column minus the "mb" column.
mm_counts_df['totals'] = mm_counts_df[col_list].sum(axis=1)

mm_counts_df.reset_index(inplace=True)
mm_counts_df = mm_counts_df.rename(columns = {'index':'value'})

#Create a table in the SQLite DB with the count data.
mm_counts_df.to_sql('mm_counts', conn, if_exists='replace', index = False)

#Print the output to confirm.
print(mm_counts_df)


    value  num_1  num_2  num_3  num_4  num_5  mb  totals
0      28     44     25     30     36     28  15     163
1      48     44     29     35     39     31   1     178
2      10     43     32     40     38     28  81     181
3      20     42     32     33     42     38  47     187
4       9     42     35     26     31     29  83     163
..    ...    ...    ...    ...    ...    ...  ..     ...
70     69      8     17     11     12     12   0      60
71     74      6      6      5      6      7   0      30
72     72      4      2      3      5      6   0      20
73     71      3      3      6      8      2   0      22
74     75      1      3      6      5     10   0      25

[75 rows x 8 columns]


### The following two code cells meet the third feature requirement:
1. Feature 3 - Make a Tableau dashboard to display your data.
    - I created a Tableau Dashboard containing bar charts of all of the results from the "mm_counts" table with Tableau Public Desktop and then I published it to the Tableau Public site.
    - Unfortunately each of the bar charts have verticle sliders which are not interactive in Visual Studio Code but they are in Google Colab.
2. Feature 3 - Make a visualization with Bokeh.

In [6]:
%%HTML
<div class='tableauPlaceholder' id='viz1679398882113' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Me&#47;MegamillionsNumbers&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='MegamillionsNumbers&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Me&#47;MegamillionsNumbers&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679398882113');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='650px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='650px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='1677px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [29]:
#Set the display theme.
curdoc().theme = 'dark_minimal'

# instantiating the figure object 
graph = figure(title = "MegaMillion Historical Winning Draw Counts")

#Create the list data.
totals = mm_counts_df['totals'].to_list()
values = mm_counts_df['value'].to_list()
  
# the points to be plotted 
x = values 
y = totals 
  
# height / thickness of the bars in the plot
width = .2
  
# plotting the verticle bar graph 
graph.vbar(x, top= y, width= width) 
  
# displaying the model 
show(graph)